<a href="https://colab.research.google.com/github/scmosoeu/fresh_produce/blob/master/colab_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Introduction

This notebook connects to the Fresh Produce Database on SQL, cleans the tables and save the cleaned tables in the database.

In [2]:
!pip install pyodbc 

     |████████████████████████████████| 276kB 3.4MB/s 
  ERROR: Failed building wheel for pyodbc
  Running setup.py clean for pyodbc
Failed to build pyodbc
    Running setup.py install for pyodbc ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-jkjvy9e5/pyodbc/setup.py'"'"'; __file__='"'"'/tmp/pip-install-jkjvy9e5/pyodbc/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-jrnfurbj/install-record.txt --single-version-externally-managed --compile Check the logs for full command output.


In [3]:
#servername: DESKTOP-TTAV4PQ\SQLEXPRESS

In [4]:
#!pip install sqlalchemy
#!conda install pymssql
#apt-get -qq install python-pymssql python3-pymssql

In [9]:
#Importing modules
import pandas as pd
import numpy as np
#import pyodbc

## Local Connection SQL

conn = pyodbc.connect('Driver={SQL Server};' 'Server=DESKTOP-TTAV4PQ\SQLEXPRESS;' 'Database=Fresh_Produce_Market_Data;' 'Trusted_Connection=yes;')

cursor = conn.cursor()

cpt_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.Capetown_Fresh_produce_market',conn) dbn_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.Durban_Fresh_produce_market',conn) jhb_comm_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.Joburg_Fresh_produce_commodity_raw',conn) jhb_con_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.Joburg_Fresh_produce_container_raw',conn) jhb_comb_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.Joburg_Fresh_produce_product_combination_raw',conn) woolworths_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.woolworths_Prices',conn) pnp_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.PickNPay_Prices',conn) shoprite_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.Shoprite_Prices',conn)

#print(sql_query)

#print(type(sql_query))

## Remote Connection (RDS)

In [10]:
# creating a connection to the remote database (RDS)
cnxn = pyodbc.connect(
    DRIVER='{SQL Server}',
    SERVER='incubate-inventory-management.cnzbp4ndrpos.eu-west-1.rds.amazonaws.com,1433',
    DATABASE='Fresh_Produce_Market_Data',
    uid='admin',
    pwd='fXToh9AXt30Yi7dGdlI2') 

NameError: ignored

In [11]:
#Creating the pandas dataframe to store the data imported from RDS instance:
cpt_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.Capetown_Fresh_produce_market',cnxn)
dbn_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.Durban_Fresh_produce_market',cnxn)
jhb_comm_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.Joburg_Fresh_produce_commodity_raw',cnxn)
jhb_con_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.Joburg_Fresh_produce_container_raw',cnxn)
jhb_comb_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.Joburg_Fresh_produce_product_combination_raw',cnxn)
woolworths_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.woolworths_Prices',cnxn)
pnp_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.PickNPay_Prices',cnxn)
shoprite_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.Shoprite_Prices',cnxn)

NameError: ignored

In [12]:
# Exploring the data
cpt_df.info()

NameError: ignored

In [13]:
#jhb_comm_df.head()
#jhb_con_df.head()
jhb_comb_df.info()

NameError: ignored

# Creating Functions to clean the data


## The function to clean Rand Value columns  and normalise them

In [14]:
def rand_value(df, column):
    """
    Takes a dataframe and a column with rand value to be cleaned as arguments, 
    returns a dataframe, with two new columns with rand value for one day and
    Month To Date rand value. The initial column that was added as argument is removed from the dataframe.
    """ 
    
    df["MTD_total_value_sold_(R)"]=np.nan
    df["total_value_sold_(R)"]=np.nan
    for i in range(len(df[column])):
        df["MTD_total_value_sold_(R)"][i]=round(float(df[column][i].split("R")[-1].replace(',','')),2)
        df["total_value_sold_(R)"][i]=round(float(df[column][i].split("R")[1].split("M")[0].replace(",","")),2)
    
    df1=df.copy()    
    df1.drop(column,axis=1, inplace=True)
    return df1

In [15]:
jhb_comm_df_cleaned=rand_value(jhb_comm_df,"total_value_sold")
jhb_con_df_cleaned=rand_value(jhb_con_df,"value_sold")

NameError: ignored

In [16]:
jhb_comm_df_cleaned.head()
jhb_con_df_cleaned.head()

NameError: ignored

## The function to clean quantities and weight columns simultaneuosly

In [17]:
def sold_qty_kg(df,quatity_sold, weight_sold):
    df["Total_quatity_sold"]=np.nan
    df["MTD_Total_quatity_sold"]=np.nan
    df["Total_kg_sold"]=np.nan
    df["MTD_total_kg_sold"]=np.nan
    for i in range(len(df[quatity_sold])):
        df["Total_quatity_sold"][i]=int(df[quatity_sold][i].split("M")[0].replace(",",""))
        df["MTD_Total_quatity_sold"][i]=int(df[quatity_sold][i].split(":")[-1].replace(',',''))
        df["Total_kg_sold"][i]=float(df[weight_sold][i].split("M")[0].replace(",",""))
        df["MTD_total_kg_sold"][i]=float(df[weight_sold][i].split(":")[-1].replace(',',''))
    df1=df.copy()
    df1.drop([quatity_sold,weight_sold],axis=1, inplace=True)
    return df1

In [18]:
jhb_comm_df_cleaned= sold_qty_kg(jhb_comm_df_cleaned, "total_qty_sold","total_kg_sold")
jhb_con_df_cleaned= sold_qty_kg(jhb_con_df_cleaned, "qty_sold","kg_sold")

NameError: ignored

In [19]:
jhb_comm_df_cleaned.head()
#jhb_con_df_cleaned.head()


NameError: ignored

In [20]:
def to_numeric(df,column):
    for i in range(len(df[column])):
        df[column][i]=df[column][i].replace(",","")
    df[column]= pd.to_numeric(df[column])
    return  df

In [21]:
cpt_df["date_updated"]=pd.to_datetime(cpt_df["date_updated"])
cpt_df=to_numeric(cpt_df,"mass")
cpt_df=to_numeric(cpt_df, "low_price")
cpt_df=to_numeric(cpt_df,"high_price")
cpt_df=to_numeric(cpt_df,"avg_price")

NameError: ignored

In [22]:
dbn_df["Date"]=pd.to_datetime(dbn_df["Date"])
dbn_df=to_numeric(dbn_df,"Weight_Kg")
dbn_df=to_numeric(dbn_df,"Low_Price")
dbn_df=to_numeric(dbn_df,"High_Price")
dbn_df=to_numeric(dbn_df,"Average_Price")
dbn_df=to_numeric(dbn_df,"Sales_Total")
dbn_df=to_numeric(dbn_df,"Total_Qty_Sold")
dbn_df=to_numeric(dbn_df,"Total_Kg_Sold")
dbn_df=to_numeric(dbn_df,"Stock_On_Hand")
dbn_df.head()

NameError: ignored

In [23]:
jhb_comb_df["date"]=pd.to_datetime(jhb_comb_df["date"])
jhb_comb_df=rand_value(jhb_comb_df, "total_value_sold")
jhb_comb_df=rand_value(jhb_comb_df, "average")
jhb_comb_df=rand_value(jhb_comb_df, "highest_price")
jhb_comb_df=rand_value(jhb_comb_df, "ave_per_kg")
jhb_comb_df=rand_value(jhb_comb_df, "highest_price_per_kg")

NameError: ignored

In [24]:
#jhb_comb_df.info()

In [25]:
woolworths_df=to_numeric(woolworths_df, "Price")
woolworths_df["Date"]=pd.to_datetime(woolworths_df["Date"])

pnp_df=to_numeric(pnp_df, "Price")
pnp_df["Date"]=pd.to_datetime(pnp_df["Date"])

shoprite_df=to_numeric(shoprite_df, "Price")
shoprite_df["Date"]=pd.to_datetime(shoprite_df["Date"])

NameError: ignored

In [26]:
# Packages need
from sqlalchemy import create_engine

import urllib

params = urllib.parse.quote_plus("DRIVER={SQL Server};SERVER=incubate-inventory-management.cnzbp4ndrpos.eu-west-1.rds.amazonaws.com,1433;DATABASE=Fresh_Produce_Market_Data;UID=admin;pwd=fXToh9AXt30Yi7dGdlI2;trusted_connection=no")
engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)





ModuleNotFoundError: ignored

Username = 'admin'

Password = 'fXToh9AXt30Yi7dGdlI2'

Host = 'incubate-inventory-management.cnzbp4ndrpos.eu-west-1.rds.amazonaws.com'

Port = '1433'
Database = 'Fresh_Produce_Market_Data'

conn = create_engine(
    'mssql+pymssql://' +
    Username + ':' + Password + '@' + Host + ':' + Port + '/' + Database
)

In [27]:
#jhb_comm_df_cleaned.to_sql('Joburg_Fresh_produce_commodity_cleaned', con=engine)
#jhb_con_df_cleaned.to_sql('Joburg_Fresh_produce_container_cleaned', con=engine)
#jhb_comb_df.to_sql('Joburg_Fresh_produce_combined_cleaned', con=engine)
cpt_df.to_sql('Capetown_Fresh_produce_market_cleaned', con=engine)
dbn_df.to_sql('Durban_Fresh_produce_market_cleaned', con=engine)
woolworths_df.to_sql('Woolworths_Prices_cleaned', con=engine)
pnp_df.to_sql('PickNPay_Prices_cleaned', con=engine)
shoprite_df.to_sql('Shoprite_Prices_cleaned', con=engine)

NameError: ignored

In [28]:
#engine.execute("SELECT * FROM Joburg_Fresh_produce_commodity_cleaned").fetchall()

In [29]:
# Check that temp table is in your MS SQL Database
conn = pyodbc.connect(driver='{SQL Server}',
                      host='DESKTOP-TTAV4PQ\SQLEXPRESS',
                      database='Fresh_Produce_Market_Data',
                      trusted_connection='true',
                      user='sa')

NameError: ignored

In [30]:
cursor.execute('SELECT * from Joburg_Fresh_produce_container_cleaned')

NameError: ignored

In [31]:
#cursor.fetchall()

Dropping a table

In [32]:
sql = 'Drop TABLE Table_testing;'
conn.execute(sql)

NameError: ignored